In [ ]:
#!pip install python-dotenv

# TODO
- Odd one out with numbers of answers
- [urgent] Word topic matching is very wrong format -> do numbers make sense here?
- There should be free text answers!
- [urgent] for gap text, implement free text answers more clearly. sometimes the gap number repeats
  - test the free text answer formatting here
  - sometimes provides correct answers in the correct order
- code the level upgrade in python logic
- code the level assessment somehow -> there should be a way to input the language level

In [1]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib

importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [2]:
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



In [3]:
profile = await db.load_profile(id='Franzy')
domain = await ui.choose_domain(domain='text')
task_type = await ui.choose_task_type(task_type='single_choice')
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [4]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate a task description, and the task according to the format of the examples. No other text or tasks.
Generate a single choice with given answer possibilities, only one of the answers is correct. Generate only one task with four answer possibilities. Generate it for the area of text teaching, targeting on Describe delicious things.
Adjust the task dificulty to language level B2 (Can understand the main ideas of complex text on both concrete and abstract topics, including technical discussions in his/her field of specialisation. Can interact with a degree of fluency and spontaneity that makes regular interaction with native speakers quite possible without strain for either party. Can produce clear, detailed text on a wide range of subjects and explain a viewpoint on a topical issue giving the advantages and disadvantages of various options).

The following are 2 examples of how the task should be generated with output format and style:

 

In [5]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Select the correct answer:
Which word best describes the taste of a perfectly ripe mango?
a) Sweet
b) Salty
c) Bitter
d) Sour


In [6]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task, answer="c")
formated_user_answer = await formater.learner_answer_formatting(user_answer)


In [7]:
######################
# This is what it will actually looks like
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

NL_feedback = Good effort! However, the correct answer is 'a) Sweet'. A perfectly ripe mango is known for its sweet taste. Keep practicing vocabulary and understanding the meaning of the question. You'll get it next time! Keep up the good work!
correctness = False
training_goals = vocabulary_goals=['mango'] grammar_goals=['adjective agreement (ripe)'] text_goals=['understanding of the question']


In [9]:
# While the parsing is not implemented, this function will not work
new_profile = await evaluator.update_learner_profile(training_goals,domain, profile, correctness)
#print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

In [10]:
print(new_profile)

name='Franzy' level=Level(vocabulary_level=5.5, grammar_level=7.9, text_level=2.9) training_goals=TrainingGoals(vocabulary_goals=['Cats and puppies', 'mango'], grammar_goals=['adjective agreement (ripe)'], text_goals=['Describe delicious things', 'understanding of the question'])


In [11]:
# This overwrites the original profile
# For testing purposes, it's easier to just comment it out
await db.save_profile(new_profile)